# *Growth Profiler* Data Analysis

The *Growth Profiler* is a high throughput device for growth characterization. Growth is automatically measured via OD in 96-well plates and the results are stored in csv files. This workflow trains analysis of growth data in csv format.

## Loading libraries
There are some general Python libraries necessary in order to have useful functions available. The library responsible for growth curve analysis is `batchslopes` [Link](https://github.com/uliebal/gp_analytics) and contains functions specifically designed for analysis of the *Growth Profiler*. 

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
import sys
sys.path.append('../..')
from iambcodes.bsfun import *
# warnings.filterwarnings("ignore", category=DeprecationWarning) 
# warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
# from collections import OrderedDict
import matplotlib.pyplot as plt

## Data input and descripion

The standard growth profiler csv file contains in the first rows some general information on the experimental conditions. We ignore these metadata in the regression analysis, however, make sure to include the header names when the real data starts. The first column should contain the time measurements, and all other columns are OD values. The *Growth Profiler* usually measures the time in minutes, other experimental data, like in *RecExpSim*, uses hours as time unit. The workflow tries to unify the time-based analysis by dividing the time vector with the variable `TimeUnit`.

The data is partitioned (binned, variable `Partition`) and in each bin a regression on the logarithmic data is conducted. The bin with the best correlation coefficient is then selected for further partitioning until the correlation coefficient gets worse (is decreasing). The correlation coefficient and the corresponding slope of the logarithmic OD are reported.

**Important**: Check the decimal separator. Science typically works with the english convention of point-separators (`10,000.23`), but the *Growth Profiler* or your local system might generate an csv with german comma-separators (`10.000,23`).


**Input:**
 - `File`: string, growth experiment csv-file name
 - `skiprows`: integer, lines of experimental metadata, Start counting from 0.
 - `decimal`: string ('.' or ','), decimal separator 
 - `TimeUnit`: integer, for measure time in minutes: 60, hours: 1
 - `Partition`: integer, decides the number of bins on which regression is performed

In [ ]:
Name = 'GP_20211027_135336_GValue.csv'        #either '../data' or a single csv e.g. 'list1.csv'

morecsv = Foldertest(Name)

if morecsv == 1:
    myFiles = Batchfilenames(Name)
else:
    myFiles = Name

myFiles

In [ ]:
#Name = 'list5.csv'       #GP_20210507_171028_MTP03_GValue

Source = 1   #1 is real GrowthProfiler, 0 is synthetic data

repnum = 3                 #number of experiments in replicates
GVexp = 1
eexp = 0

Partition = 4

FileName = 'list3.csv'
WellID = 'A1'

if not isinstance(myFiles, str):
    index = [idx for idx, s in enumerate(myFiles) if FileName in s][0]
else:
    File = myFiles

myparams = Fileparams(Source)

#pd.options.display.max_rows

if isinstance(myFiles, str):
    df = pd.read_csv(File, skiprows=myparams['skiprows'] , decimal=myparams['decimal'], sep = ",", index_col='Time (min)')
else:
    print('{}, {}'.format(FileName, WellID))
    df = pd.read_csv(myFiles[index][5:], skiprows=myparams['skiprows'] , decimal=myparams['decimal'], sep = ",", index_col='Time (min)')
    

if 'Input_Image' in list(df.columns):
    df = df.drop(labels='Input_Image', axis=1)
df.index = df.index/myparams['TimeUnit']
OD = pd.DataFrame()
myCols = df.columns
for i1 in myCols: 
    OD[i1] = CorrectedOD(df[i1], GVexp, eexp)
TimeAx = df.index #'Time (min)' # for growth profiler
Exp0 = df.index

for column in df.columns:
    if 'Unnamed' in column:
        df.drop(labels= column, axis=1, inplace=True)
        
# print('Column headers: ', df.columns)
# df.plot(x=TimeAx, y=Exp0)
df[WellID].plot()


#plt.savefig('myPlots.svg', format='svg')

## All Experiment Analysis

Below, all experiments in the csv file are analysed. The final information is stored in plots showing the regression range for each experiment along with the growth rate and the correlation coefficient and stored as `myPlots.svg`. If the regressions look unconvincing, try a different partition.

By doubleclicking on the subplots, you can zoom in for better visual recognition.

In this next section, analyzed data will be stored as Excel file(s) in the same folder, too.

In [ ]:
if isinstance(myFiles, str):

    myCols = df.columns
    subplot_x = round(np.sqrt(len(myCols)))
    subplot_y = round(np.sqrt(len(myCols))) + 1
    
#     myr2table = pd.DataFrame()
    mu_list = []
    r2_list = []

    NumExp = len(myCols)
    AxDim = np.ceil(np.sqrt(NumExp))
    t = df.index.values
# plt.subplots(AxDim, AxDim, sharex='col')
    fig, ax = plt.subplots(figsize=[20,10], sharey=True)
    print(myCols)
    for idx, myExp in enumerate(myCols):
#print(myExp)
        x = OD[myExp].values
        myResult = DetectR2MaxSingle(t,x,Partition)
        plt.subplot(subplot_x, subplot_y, idx+1)
        plt.plot(t, np.log(x))
        plt.title(myExp)
        if myResult is not False:
            plt.scatter(myResult['time'], np.log(myResult['OD']))
            if Source == 1:
                plt.plot(myResult['time'], myResult['Slope']*myResult['time'] + myResult['ycorrect'], 'r', label='$\mu$:{:.3f}, R2:{:.2f}'.format(myResult['Slope']*60, myResult['R2']))
            else:
                plt.plot(myResult['time'], myResult['Slope']*myResult['time'] + myResult['ycorrect'], 'r', label='$\mu$:{:.3f}, R2:{:.2f}'.format(myResult['Slope'], myResult['R2']))
            myResult['ID'] = myExp
            plt.legend()
        #hier Dataframe für Mu und R2 feeden
            mu_list.append(myResult['Slope'])
            r2_list.append(myResult['R2'])
        if myResult is False:
            mu_list.append(False)
            r2_list.append(False)
        plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    
    fig.text(0.5, 0.1, 'time', ha='center')
    fig.text(0.1, 0.5, 'ln(OD)', va='center', rotation='vertical')
    means, stdev = StatsCalc(OD, repnum)
    tk = 1
    createExcelFiles(tk, means, stdev, myCols, mu_list, r2_list, repnum, df, OD, myFiles)
else:
    df = {}
    mu = {}
    r2 = {}
    myr2table = pd.DataFrame()
    with pd.ExcelWriter('myr2table.xlsx') as writer:  
        myr2table.to_excel(writer, sheet_name='x')
    myr2table = pd.DataFrame()
    for i in range(len(myFiles)):
        df[i] = pd.read_csv(myFiles[i][5:], skiprows=myparams['skiprows'] , decimal=myparams['decimal'], sep = ",", index_col = 'Time (min)')
        if 'Input_Image' in df[i].columns:
            df [i] = df[i].drop(labels='Input_Image', axis=1)
        if 'Unnamed: 25' in df[i].columns:
            df [i] = df[i].drop(labels='Unnamed: 25', axis=1)
        myCols = df[i].columns
        OD = pd.DataFrame()
        for i1 in myCols:
            OD = CorrectedOD(df[i], GVexp, eexp)
        mu_list = []
        r2_list = []
        NumExp = len(myCols)
        AxDim = np.ceil(np.sqrt(NumExp))
        t = df[i].index.values
        for idx, myExp in enumerate(myCols):
            x = OD[myExp].values
            myResult = DetectR2MaxSingle(t,x,Partition)
            if myResult is not False:
                myResult['ID'] = myExp
                mu_list.append(myResult['Slope'])
                r2_list.append(myResult['R2'])
            if myResult is False:
                mu_list.append(False)
                r2_list.append(False)
        mu[i] = mu_list
        r2[i] = r2_list
        myr2table['Wells'] = myCols
        myr2table['Mu value'] = mu[i]
        myr2table['R2 value'] = r2[i]
        
    means, stdev = StatsCalc(OD, repnum)
    for tk in range(len(myFiles)):
        createExcelFiles(tk, means, stdev, myCols, mu_list, r2_list, repnum, df, OD, myFiles)

## Single Experiment Analysis

Below, we check individual growth curves and their regression. This can be helpful if you want to examine a particular experiment.

In [ ]:
t = df.index.values
x = df[WellID].values  # instead of WellID, one can as well directly search after 'A1', 'B3', etc. for example. 


myResult = DetectR2MaxSingle(t,x,Partition)
# print(myResult)
plt.plot(t, np.log(x))
if myResult!=False:
    plt.scatter(myResult['time'], np.log(myResult['OD']))
    plt.plot(myResult['time'], myResult['Slope']*myResult['time'] + myResult['ycorrect'], 'r', label='$\mu$:{:.3f}, R2:{:.2f}'.format(myResult['Slope'], myResult['R2']))
    plt.legend()
    myResult['ID'] = Exp0
# myResult

In [ ]:
Yield = 0.78
InitSubsConc = 50
sample = pd.DataFrame()
sample = df.iloc[:, 0]
mylist = [InitSubsConc]
for indx in range(1, len(df['A1'])-1):
    BiomassDiff = df.loc[df.index[indx], 'A1']-df.loc[df.index[indx-1], 'A1']
    if BiomassDiff < 0:
        BiomassDiff = 0
    SubsUptake = mylist[-1]-(BiomassDiff/Yield)
    mylist.append(SubsUptake)
mylist.append(SubsUptake)
mylist
columnsample = pd.DataFrame({'Substrate Uptake':mylist}, index = df.index)
# columnsample.plot()
# sample['Substrate'] = mylist
sample = pd.concat([sample, columnsample], axis=1)
# type(sample)
sample.plot()